In [9]:
import re
import ast
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import os

In [10]:
# Paths
DATA_DIR = 'data'
COORD_FILE = os.path.join(DATA_DIR, 'instance_coordinates.txt')
OUT_TXT = 'output.txt'
OUT_DIR = 'plots'

os.makedirs(OUT_DIR, exist_ok=True)

In [11]:
# 1. Load coordinates
df_coords = pd.read_csv(
    COORD_FILE,
    sep=r'\s+',
    header=None,
    names=['id', 'x', 'y']
)
coords = df_coords.set_index('id')[['x', 'y']]

In [12]:
# 2. Parse routes
pattern = re.compile(
    r"^Vehicle\s+\d+\s+route\s+\(Distance:[^\)]+\):\s*(\[[^\]]+\])"
)
routes = []
with open(OUT_TXT, 'r') as f:
    for line in f:
        m = pattern.match(line)
        if m:
            routes.append(ast.literal_eval(m.group(1)))

In [13]:
# 3. Generate a unique color for each route via HSV
num_routes = len(routes)
hsv_cmap = matplotlib.colormaps['hsv']
colors = [hsv_cmap(i / num_routes) for i in range(num_routes)]

In [14]:
# 4. Combined plot
plt.figure(figsize=(10, 10))
plt.scatter(coords['x'], coords['y'], c='lightgrey', s=30, label='Customers')
depot = coords.loc[1]
plt.scatter(depot.x, depot.y, c='red', s=120, marker='*', label='Depot')

for idx, route in enumerate(routes):
    pts = np.array([coords.loc[node + 1] for node in route])
    plt.plot(
        pts[:, 0], pts[:, 1],
        marker='o', linestyle='-',
        linewidth=1, markersize=4,
        color=colors[idx],
        label=f'Vehicle {idx + 1}'
    )

plt.title('All VRPTW Routes')
plt.xlabel('X coordinate')
plt.ylabel('Y coordinate')
plt.axis('equal')
plt.grid(True)
plt.legend(fontsize='small', ncol=2, loc='upper right')
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, 'all_routes.png'), dpi=300)
plt.close()

In [15]:
# 5. Separate plots
for idx, route in enumerate(routes):
    plt.figure(figsize=(8, 8))
    plt.scatter(coords['x'], coords['y'], c='lightgrey', s=20)
    plt.scatter(depot.x, depot.y, c='red', s=100, marker='*')

    pts = np.array([coords.loc[node + 1] for node in route])
    plt.plot(
        pts[:, 0], pts[:, 1],
        marker='o', linestyle='-',
        linewidth=1.5, markersize=6,
        color=colors[idx]
    )

    plt.title(f'Vehicle {idx + 1} Route')
    plt.xlabel('X coordinate')
    plt.ylabel('Y coordinate')
    plt.axis('equal')
    plt.grid(True)
    plt.tight_layout()
    filename = f'route_{idx + 1}.png'
    plt.savefig(os.path.join(OUT_DIR, filename), dpi=300)
    plt.close()

print(f"Saved combined plot to {OUT_DIR}/all_routes.png")
print(f"Saved {len(routes)} individual route plots to {OUT_DIR}/route_<veh_id>.png")

Saved combined plot to plots/all_routes.png
Saved 51 individual route plots to plots/route_<veh_id>.png
